Import libraries

In [1]:
from keras.utils import to_categorical
from keras.models import load_model
from sklearn.model_selection import StratifiedKFold
import numpy as np
import pandas as pd
from sklearn.metrics import precision_recall_fscore_support, accuracy_score, cohen_kappa_score, confusion_matrix
from utils import load_data, AA_andEachClassAccuracy
import keras.backend as k
import tensorflow as tf

k.set_image_data_format('channels_last')

Load Dataset

In [2]:
# Select dataset. Options are 'Kochia', 'IP', 'PU', 'SA', or 'EUROSAT'
dataset = 'IP'
train_x, train_y = load_data(dataset=dataset, test=True)
if dataset == 'Kochia':
    classes = 3
elif dataset == 'EUROSAT':
    classes = 10
else:
    classes = int(np.max(train_y)) + 1
print("Dataset correctly imported")

Dataset correctly imported


PREDICT AND CALCULATE METRICS

In [3]:
windowSize = train_x.shape[1]
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=7)
cvoa = []
cvaa = []
cvka = []
cvpre = []
cvrec = []
cvf1 = []

# Select Network. Options are 'hyper3dnet', 'hybridsn', 'spectrum',
# 'resnet50' (expect for Kochia), or 'kochiafc' (only for Kochia)
network = 'hyper3dnet'

# Initialize
confmatrices = np.zeros((10, int(classes), int(classes)))

ntrain = 1
model = None
for train, test in kfold.split(train_x, train_y):

    k.clear_session()

    ytest = to_categorical(train_y[test], num_classes=classes).astype(np.int32)

    # Load network and weights of the 'ntrain'-fold
    model = load_model("weights/" + dataset + "/" + network + "/weights-" + network + dataset + str(ntrain) + ".h5")
    model.trainable = False

    # Predict results
    ypred = model.predict(train_x[test])

    # Calculate confusion matrix
    sess = tf.compat.v1.Session()
    with sess.as_default():
        con_mat = tf.math.confusion_matrix(labels=np.argmax(ytest, axis=-1),
                                           predictions=np.argmax(ypred, axis=-1)).numpy()
    con_mat_norm = np.around(con_mat.astype('float') / con_mat.sum(axis=1)[:, np.newaxis], decimals=3)
    classes_list = list(range(0, int(classes)))
    con_mat_df = pd.DataFrame(con_mat_norm, index=classes_list, columns=classes_list)
    confmatrices[ntrain - 1, :, :] = con_mat_df.values

    # Calculate metrics
    oa = accuracy_score(np.argmax(ytest, axis=1), np.argmax(ypred, axis=-1))
    confusion = confusion_matrix(np.argmax(ytest, axis=1), np.argmax(ypred, axis=-1))
    each_acc, aa = AA_andEachClassAccuracy(confusion)
    kappa = cohen_kappa_score(np.argmax(ytest, axis=1), np.argmax(ypred, axis=-1))
    prec, rec, f1, support = precision_recall_fscore_support(np.argmax(ytest, axis=1), np.argmax(ypred, axis=-1),
                                                             average='macro')

    # Add metrics to the list
    cvoa.append(oa * 100)
    cvaa.append(aa * 100)
    cvka.append(kappa * 100)
    cvpre.append(prec * 100)
    cvrec.append(rec * 100)
    cvf1.append(f1 * 100)

    ntrain = ntrain + 1


print("Overall accuracy: %.3f%% (+/- %.3f%%)" % (float(np.mean(cvoa)), float(np.std(cvoa))))
print("Average accuracy: %.3f%% (+/- %.3f%%)" % (float(np.mean(cvaa)), float(np.std(cvaa))))
print("Kappa accuracy: %.3f%% (+/- %.3f%%)" % (float(np.mean(cvka)), float(np.std(cvka))))
print("Precision accuracy: %.3f%% (+/- %.3f%%)" % (float(np.mean(cvpre)), float(np.std(cvpre))))

Overall accuracy: 99.942% (+/- 0.064%)
Average accuracy: 99.901% (+/- 0.181%)
Kappa accuracy: 99.933% (+/- 0.073%)
Precision accuracy: 99.840% (+/- 0.239%)


Network Architecture

In [4]:
model.summary()

Model: "model_11"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_11 (InputLayer)           [(None, 25, 25, 30,  0                                            
__________________________________________________________________________________________________
conv3d_41 (Conv3D)              (None, 25, 25, 30, 8 512         input_11[0][0]                   
__________________________________________________________________________________________________
batch_normalization_81 (BatchNo (None, 25, 25, 30, 8 32          conv3d_41[0][0]                  
__________________________________________________________________________________________________
activation_81 (Activation)      (None, 25, 25, 30, 8 0           batch_normalization_81[0][0]     
___________________________________________________________________________________________